# 🪟 BÀI TẬP 4: WINDOW FUNCTIONS

## Mục tiêu học tập:
- ROW_NUMBER, RANK, DENSE_RANK
- LAG, LEAD
- FIRST_VALUE, LAST_VALUE
- Running totals (SUM OVER)
- Moving averages
- PARTITION BY
- Frame clauses (ROWS BETWEEN)

In [ ]:
# Setup
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def get_connection():
    return psycopg2.connect(
        host='postgres',
        port=5432,
        database='data_engineer',
        user='dataengineer',
        password='dataengineer123'
    )

def run_query(query):
    conn = get_connection()
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

print("✅ Setup ready!")

---
## 🎯 EXERCISE 1: ROW_NUMBER

**Câu hỏi:** Rank customers theo total revenue

In [ ]:
# YOUR CODE HERE
query = """
SELECT 
    c.customer_name,
    SUM(o.total_amount) as total_revenue,
    ROW_NUMBER() OVER (ORDER BY SUM(o.total_amount) DESC) as rank
FROM analytics.customers c
INNER JOIN analytics.orders o ON c.customer_id = o.customer_id
WHERE o.order_status = 'completed'
GROUP BY c.customer_id, c.customer_name
ORDER BY rank
LIMIT 20;
"""

result = run_query(query)
result

---
## 🎯 EXERCISE 2: RANK vs DENSE_RANK

**Câu hỏi:** So sánh RANK và DENSE_RANK cho products theo price

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: SELECT với cả RANK() và DENSE_RANK()
"""

result = run_query(query)
result.head(20)

---
## 🎯 EXERCISE 3: PARTITION BY

**Câu hỏi:** Rank products theo price TRONG MỖI category

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC)
"""

result = run_query(query)
result

---
## 🎯 EXERCISE 4: LAG và LEAD

**Câu hỏi:** Tính revenue growth month-over-month

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: LAG(revenue) OVER (ORDER BY month)
"""

result = run_query(query)
result

---
## 🎯 EXERCISE 5: Running total

**Câu hỏi:** Tính cumulative revenue theo ngày

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: SUM(revenue) OVER (ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
"""

result = run_query(query)
result.head(20)

---
## 🎯 EXERCISE 6: Moving average

**Câu hỏi:** Tính 7-day moving average của daily revenue

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: AVG(revenue) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
"""

result = run_query(query)
result.head(20)

---
## 🎯 EXERCISE 7: FIRST_VALUE và LAST_VALUE

**Câu hỏi:** Tìm first và last order của mỗi customer

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
"""

result = run_query(query)
result.head(20)

---
## 🎯 CHALLENGE: Customer RFM Analysis

**Câu hỏi:** Tính RFM scores:
- Recency: Days since last order
- Frequency: Number of orders
- Monetary: Total revenue

Rank customers (1-5) cho mỗi metric

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: NTILE(5) OVER (ORDER BY metric)
"""

result = run_query(query)
result.head(20)

---
## 📊 VISUALIZATION: Running total

In [ ]:
query = """
WITH daily_revenue AS (
    SELECT 
        order_date::date as date,
        SUM(total_amount) as daily_revenue
    FROM analytics.orders
    WHERE order_status = 'completed'
    GROUP BY order_date::date
)
SELECT 
    date,
    daily_revenue,
    SUM(daily_revenue) OVER (ORDER BY date) as cumulative_revenue
FROM daily_revenue
ORDER BY date;
"""

df = run_query(query)
df['date'] = pd.to_datetime(df['date'])

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df['date'], df['cumulative_revenue'], linewidth=2, color='#2E86AB')
ax.fill_between(df['date'], df['cumulative_revenue'], alpha=0.3, color='#2E86AB')
ax.set_title('Cumulative Revenue Over Time', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Revenue ($)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 📚 SOLUTIONS

In [ ]:
# # SOLUTION 4: LAG - Month-over-month growth
# query = """
# WITH monthly_revenue AS (
#     SELECT 
#         DATE_TRUNC('month', order_date) as month,
#         SUM(total_amount) as revenue
#     FROM analytics.orders
#     WHERE order_status = 'completed'
#     GROUP BY DATE_TRUNC('month', order_date)
# )
# SELECT 
#     month,
#     revenue,
#     LAG(revenue) OVER (ORDER BY month) as prev_month_revenue,
#     revenue - LAG(revenue) OVER (ORDER BY month) as revenue_change,
#     ROUND(100.0 * (revenue - LAG(revenue) OVER (ORDER BY month)) / 
#           NULLIF(LAG(revenue) OVER (ORDER BY month), 0), 2) as growth_pct
# FROM monthly_revenue
# ORDER BY month;
# """

# # SOLUTION 8: RFM Analysis
# query = """
# WITH customer_metrics AS (
#     SELECT 
#         c.customer_id,
#         c.customer_name,
#         CURRENT_DATE - MAX(o.order_date::date) as recency_days,
#         COUNT(o.order_id) as frequency,
#         SUM(o.total_amount) as monetary
#     FROM analytics.customers c
#     INNER JOIN analytics.orders o ON c.customer_id = o.customer_id
#     WHERE o.order_status = 'completed'
#     GROUP BY c.customer_id, c.customer_name
# )
# SELECT 
#     customer_name,
#     recency_days,
#     frequency,
#     monetary,
#     NTILE(5) OVER (ORDER BY recency_days) as r_score,
#     NTILE(5) OVER (ORDER BY frequency DESC) as f_score,
#     NTILE(5) OVER (ORDER BY monetary DESC) as m_score
# FROM customer_metrics
# ORDER BY monetary DESC
# LIMIT 50;
# """